In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/AIFFEL_quest_rs/Exploration/Quest01/
!ls

/content/drive/MyDrive/AIFFEL_quest_rs/Exploration/Quest01
data		Pokemon.csv	pytoch_5_DatdAug.ipynb		  Quest01.ipynb
datasets	pytoch_2.ipynb	pytoch_6_ImgClassification.ipynb  README.md
models		pytoch_3.ipynb	pytoch_7.ipynb
non_linear.csv	pytoch_4.ipynb	pytorch_1.ipynb


In [ ]:
!git pull origin main

From https://github.com/nagujean/AIFFEL_quest_rs
 * branch            main       -> FETCH_HEAD
Auto packing the repository in background for optimum performance.
See "git help gc" for manual housekeeping.
and remove .git/gc.log
Automatic cleanup will not be performed until the file is removed.


Already up to date.


In [1]:
!git add .

In [2]:
  !git config --global user.email "nagujean@gmail.com"
  !git config --global user.name "nagujean"

In [3]:
!git commit -m "commit"

[main 72e1668] commit
 3 files changed, 1497 insertions(+), 2 deletions(-)
 create mode 100644 Exploration/Quest01/.ipynb_checkpoints/pytoch_7-checkpoint.ipynb
 create mode 100644 Exploration/Quest01/colab_test.ipynb


In [ ]:
!git push origin main

Username for 'https://github.com': 

In [ ]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torch.utils.data import random_split

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


train_dataset = datasets.Imagenette(root="data", split="train", download=True, transform=transform)
val_dataset = datasets.Imagenette(root="data", split="val", download=True, transform=transform)

val_size = len(val_dataset)
print(val_size)
test_size = val_size // 2
print(test_size)
new_val_size = val_size - test_size
print(new_val_size)

val_dataset, test_dataset = random_split(val_dataset, [new_val_size, test_size], generator=torch.Generator().manual_seed(2025))

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

3925
1962
1963


In [ ]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([32, 3, 224, 224])
Shape of y: torch.Size([32]) torch.int64


In [ ]:
# 학습에 사용할 CPU나 GPU, MPS 장치를 얻습니다.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 모델을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3*224*224, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=150528, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def val(dataloader, model, loss_fn):
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            val_loss += loss_fn(pred, y).item()

            correct += (pred.argmax(1) == y).sum().item()
            total += y.size(0)

    avg_loss = val_loss / len(dataloader)
    accuracy = correct / total * 100
    print(f"Validation loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%\n")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    val(val_dataloader, model, loss_fn)
    test(test_dataloader, model, loss_fn)
print("Done!")



Epoch 1
-------------------------------
loss: 2.273432  [   32/ 9469]
loss: 1.845385  [ 3232/ 9469]
loss: 1.892853  [ 6432/ 9469]
Validation loss: 1.8080, Accuracy: 40.04%

Test Error: 
 Accuracy: 38.1%, Avg loss: 1.798778 

Epoch 2
-------------------------------
loss: 1.681618  [   32/ 9469]
loss: 1.675129  [ 3232/ 9469]
loss: 1.717267  [ 6432/ 9469]
Validation loss: 1.7182, Accuracy: 41.16%

Test Error: 
 Accuracy: 41.3%, Avg loss: 1.724608 

Epoch 3
-------------------------------
loss: 1.554750  [   32/ 9469]
loss: 1.328110  [ 3232/ 9469]
loss: 1.568715  [ 6432/ 9469]
Validation loss: 1.6810, Accuracy: 43.66%

Test Error: 
 Accuracy: 43.0%, Avg loss: 1.683282 

Epoch 4
-------------------------------
loss: 1.507005  [   32/ 9469]
loss: 1.533525  [ 3232/ 9469]
loss: 1.608364  [ 6432/ 9469]
Validation loss: 1.6453, Accuracy: 44.68%

Test Error: 
 Accuracy: 44.6%, Avg loss: 1.645028 

Epoch 5
-------------------------------
loss: 1.231570  [   32/ 9469]
loss: 1.698684  [ 3232/ 9469]


In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

In [ ]:
classes = [
    "tench",
    "English springer",
    "cassette player",
    "chain saw",
    "church",
    "French horn",
    "garbage truck",
    "gas pump",
    "golf ball",
    "parachute",
]




model.eval()
x, y = test_dataset[0][0], test_dataset[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')